# HBV model

This notebook can be used to run your HBV model. It imports the packages needed to run the HBV model on eWaterCycle. 

**Important:**
If you edited the code in hbv_bmi, it is important to restart the kernel and re-run the cells. 


In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# eWaterCycle packages
import ewatercycle
from forcing import HBVForcing
from ewatercycle.base.forcing import GenericLumpedForcing

In [3]:
from ewatercycle_wrapper_HBV import HBV

Important  to set paths well:

In [4]:
shape = Path(ewatercycle.__file__).parent / "testing/data/Rhine/Rhine.shp"
path = Path.cwd()
forcing_path = path / "Forcing"

specify dates

In [5]:
experiment_start_date = "1997-08-01T00:00:00Z"
experiment_end_date = "2000-08-31T00:00:00Z"


setup forcing

In [6]:
camels_forcing = HBVForcing(start_time = experiment_start_date,
                          end_time = experiment_end_date,
                          directory = forcing_path,
                          evspsblpot = f'camels_01022500.nc',
                          pr = f'camels_01022500.nc'
                          )

In [7]:
model = HBV(forcing=camels_forcing)

parameter information

In [8]:
s_0 = np.array([0,  100,  0,  5])
p_min_initial= np.array([0,   0.2,  40,    .5,   .001,   1,     .01,  .0001])
p_max_initial = np.array([8,    1,  800,   4,    .3,     10,    .1,   .01])
p_names = ["$I_{max}$",  "$C_e$",  "$Su_{max}$", "β",  "$P_{max}$",  "$T_{lag}$",   "$K_f$",   "$K_s$"]
S_names = ["Interception storage", "Unsaturated Rootzone Storage", "Fastflow storage", "Groundwater storage"]
param_names = ["Imax","Ce",  "Sumax", "beta",  "Pmax",  "Tlag",   "Kf",   "Ks"]
par_0 = (p_min_initial + p_max_initial)/2

In [9]:
config_file, _ = model.setup(
                            parameters=','.join([str(p) for p in par_0]),
                            initial_storage=','.join([str(s) for s in s_0]),
                               )

RuntimeError: NetCDF: Name contains illegal characters: (variable '"evspsblpot"', group '/')

In [ ]:
model.initialize(config_file)

In [ ]:
Q_m = []
time = []
Pe = []
Si = []
Imax = []
Ei = []
Ep = []
P = []
while model.time < model.end_time:
    model.update()
    Q_m.append(model.get_value("Q"))
    time.append(pd.Timestamp(model.time_as_datetime.date()))

In [ ]:
df = pd.DataFrame(data=Q_m,columns=["Modeled discharge"],index=time)

In [ ]:
Q_ref = np.loadtxt('Forcing/Q_m_out_ref.txt')

In [ ]:
fig, ax = plt.subplots(1,1)
df.plot(ax=ax,label="Modeled discharge HBV-bmi")
plt.plot(time, Q_ref[1:],label="Ref discharge HBV-bmi")
ax.legend(bbox_to_anchor=(1,1));

In [ ]:
model.finalize()

### Check wether your code is correct
To test your HBV model, run the next cell. This is in example with different parameters and initial storages, to see if your implemented function works properly. 

In [ ]:
# model_test = HBV(forcing=test_forcing)
# parameters_test = np.array([4,   0.5,  200,    2.5,   .1,   5,     .01,  .0001])
# initial_storage_test = np.array([100,  0,  0,  0])
# config_file_test, _ = model_test.setup(
#                             parameters=','.join([str(p) for p in parameters_test]),
#                             initial_storage=','.join([str(s) for s in initial_storage_test]),
#                                )
# model_test.initialize(config_file_test)

# model_test.update()

# assert np.isclose(model_test.get_value('Pe_dt'), 104.888, rtol=1e-4, atol=1e-4)
# assert np.isclose(model_test.get_value('Ei_dt'), 0, rtol=1e-4, atol=1e-4)
# assert np.isclose(model_test.get_value('Ea_dt'), 1.79891224, rtol=1e-4, atol=1e-4)

# assert np.isclose(model_test.get_value('Si'), 4, rtol=1e-4, atol=1e-4)
# assert np.isclose(model_test.get_value('Sf'), 0, rtol=1e-4, atol=1e-4)
# assert np.isclose(model_test.get_value('Su'), 103.03754322, rtol=1e-4, atol=1e-4)
# assert np.isclose(model_test.get_value('Ss'), 0.05153939, rtol=1e-4, atol=1e-4)

# assert np.isclose(model_test.get_value("Qus_dt"), 0.05154454, rtol=1e-4, atol=1e-4)
# assert np.isclose(model_test.get_value('Qf_dt'), 0, rtol=1e-4, atol=1e-4)
# assert np.isclose(model_test.get_value('Qs_dt'), 5.15445439e-06, rtol=1e-4, atol=1e-4)

# model_test.finalize()